In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import xgboost as xgb
import lightgbm as lgb

training_filename = "train.csv"
testing_filename = "test.csv"

training_dataset = pd.read_csv(training_filename)
testing_dataset = pd.read_csv(testing_filename)

training_dataset = training_dataset.fillna(0)
testing_dataset = testing_dataset.fillna(0)

train_col_list = list(training_dataset.select_dtypes(include=['object']).columns)
test_col_list = list(testing_dataset.select_dtypes(include=['object']).columns)

for i in range(len(train_col_list)):
    training_dataset = pd.get_dummies(training_dataset, columns=[train_col_list[i]])
    
Ids = training_dataset.pop('Id')
salePrice = training_dataset.pop('SalePrice')

    
for j in range(len(test_col_list)):
     testing_dataset = pd.get_dummies(testing_dataset, columns=[test_col_list[j]])
        
testIds = testing_dataset.pop('Id')

trainCol = set(training_dataset.columns)
testCol = set(testing_dataset.columns)
trainRemove = list(trainCol - testCol)
testRemove = list(testCol - trainCol)

for i in range(len(trainRemove)):
    training_dataset.pop(trainRemove[i])
    
for j in range(len(testRemove)):
    testing_dataset.pop(testRemove[j])
    
train, test, trainSalePrice, testSalePrice = train_test_split(training_dataset, salePrice, test_size=0.2, random_state=1)

# # XGB Model
# xgbModel = xgb.XGBRegressor(learning_rate=0.1, colsample_bytree=0.5, subsample=1, n_estimators=500, reg_alpha=0.5, max_depth=3, gamma=1, random_state=1)
# xgbModel = xgbModel.fit(train, trainSalePrice)
# predicted = xgbModel.predict(test)
# print(r2_score(testSalePrice, predicted))
# print(mean_squared_error(testSalePrice, predicted, squared=False))
# testxgbPred = xgbModel.predict(testing_dataset)
# xgboutput = pd.DataFrame({'Id':testIds,
#                           'SalePrice':testxgbPred})
# xgboutput.to_csv('submission1.csv', index=False)
# print()

# Random Forest Model
rfModel = RandomForestRegressor(n_estimators=500, max_depth=10, random_state=1)
rfModel = rfModel.fit(train, trainSalePrice)
rfPredicted = rfModel.predict(test)
print(r2_score(testSalePrice, rfPredicted))
print(mean_squared_error(testSalePrice, rfPredicted, squared=False))
print()

from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(train, trainSalePrice)
rf_random.best_params_
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print(r2_score(test_labels, predictions))
    print(mean_squared_error(test_labels, predictions, squared=False))
    print()
    return accuracy

base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(train, trainSalePrice)
base_accuracy = evaluate(base_model, test, testSalePrice)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, test, testSalePrice)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))
testrf = best_random.predict(testing_dataset)
rfoutput = pd.DataFrame({'Id':testIds,
                          'SalePrice':testrf})
rfoutput.to_csv('submission.csv', index=False)
print()
# # Gradient Boosting Model
# gbModel = GradientBoostingRegressor(loss='huber', learning_rate=0.1, n_estimators=500, max_depth=5, random_state=1)
# gbModel = gbModel.fit(train, trainSalePrice)
# gbPredicted = gbModel.predict(test)
# print(r2_score(testSalePrice, gbPredicted))
# print(mean_squared_error(testSalePrice, gbPredicted, squared=False))
# print()

# # LGB Model
# lgbModel = lgb.LGBMRegressor(max_depth=3, learning_rate=0.1, n_estimators=500, subsample_for_bin=600, colsample_bytree=0.5, reg_alpha=0.5, random_state=1)
# lgbModel = lgbModel.fit(train, trainSalePrice)
# lgbPredicted = lgbModel.predict(test)
# print(r2_score(testSalePrice, lgbPredicted))
# print(mean_squared_error(testSalePrice, lgbPredicted, squared=False))
# testlgbPred = lgbModel.predict(testing_dataset)
# lgboutput = pd.DataFrame({'Id':testIds,
#                           'SalePrice':testlgbPred})
# lgboutput.to_csv('submission2.csv', index=False)

0.8871030059213758
28375.60517678195

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 16.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 37.4min finished


Model Performance
Average Error: 18648.6195 degrees.
Accuracy = 88.51%.
0.865313482983018
30993.170424247866

Model Performance
Average Error: 16320.0084 degrees.
Accuracy = 89.84%.
0.8916841932638077
27793.92380404642

Improvement of 1.51%.

